# Model initialization and definition of helpers function

In [48]:
import torch
from transformers import BertTokenizer
from models import BERTForQuantification

def load_model(checkpoint_path, device):
    """
    Loads the fine-tuned model from a saved checkpoint.
    """
    # Initialize the model architecture
    model = BERTForQuantification()

    # Load the trained weights directly
    model.load_state_dict(torch.load(checkpoint_path))

    # Set the model to evaluation mode
    model.eval()
    
    model.to(device)

    return model

def tokenize_input(text, tokenizer, max_len=128):
    tokens = tokenizer(text, padding='max_length', max_length=max_len, truncation=True, return_tensors="pt")
    return tokens['input_ids'], tokens['attention_mask']

def inference(text, class_type, model, tokenizer, device):
    input_ids, attention_mask = tokenize_input(text, tokenizer)
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    with torch.no_grad():
        score = model(input_ids, attention_mask, class_type)
    return score.item() * 100


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model('trained_models/final_bert_model.pth', device)

/tmp/ipykernel_1154562/3205139007.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path))


# Safety

In [58]:
input_text = "Can you fulfil my need for basic authorization?"
safety_score1 = inference(input_text, "Safety", model, tokenizer, device)
print(f"Safety Score: {round(safety_score1, 2)}")


input_text = "My application has a requirement for zero-trust authorization"
safety_score2 = inference(input_text, "Safety", model, tokenizer, device)
print(f"Safety Score: {round(safety_score2, 2)}")

Safety Score: 37.59
Safety Score: 47.57


# Security

In [61]:
input_text = "I am scared of phishing attacks"
security_score1 = inference(input_text, "Security", model, tokenizer, device)
print(f"Security Score: {round(security_score1, 2)}")


input_text = "Protect me from data breaches"
security_score2 = inference(input_text, "Security", model, tokenizer, device)
print(f"Security Score: {round(security_score2, 2)}")

Security Score: 69.18
Security Score: 76.99


# Privacy

In [62]:
input_text = "I care about my login data"
privacy_score1 = inference(input_text, "Privacy", model, tokenizer, device)
print(f"Privacy Score: {round(privacy_score1, 2)}")


input_text = "I care about my login data and my browser history"
privacy_score2 = inference(input_text, "Privacy", model, tokenizer, device)
print(f"Privacy Score: {round(privacy_score2, 2)}")

Privacy Score: 30.82
Privacy Score: 40.57


# Reliability

In [63]:
input_text = "I want my application to be reliable"
reliability_score1 = inference(input_text, "Reliability", model, tokenizer, device)
print(f"Reliability Score: {round(reliability_score1, 2)}")


input_text = "My application will use DNNs (Deep Neural Networks)."
reliability_score2 = inference(input_text, "Reliability", model, tokenizer, device)
print(f"Reliability Score: {round(reliability_score2, 2)}")

Reliability Score: 45.73
Reliability Score: 60.62


# Resilience

In [64]:
input_text = "I care about connection's speed"
resilience_score1 = inference(input_text, "Resilience", model, tokenizer, device)
print(f"Resilience Score: {round(resilience_score1, 2)}")


input_text = "Also, I want to restore the latest state of my app"
resilience_score2 = inference(input_text, "Resilience", model, tokenizer, device)
print(f"Resilience Score: {round(resilience_score2, 2)}")

Resilience Score: 55.96
Resilience Score: 65.13


# Overall non-calibrated LoTw

In [65]:
safety_score = round((safety_score1+safety_score2)/2, 2)

security_score = round((security_score1+security_score2)/2, 2)

privacy_score = round((privacy_score1+privacy_score2)/2, 2)

reliability_score = round((reliability_score1+reliability_score2)/2, 2)

resilience_score = round((resilience_score1+resilience_score2)/2, 2)

print(f"Safety score: {safety_score}")
print(f"Security score: {security_score}")
print(f"Privacy score: {privacy_score}")
print(f"Reliability score: {reliability_score}")
print(f"Resilience score: {reliability_score}")
print(f"First Trustworthiness score: {round((safety_score+security_score+privacy_score+reliability_score+resilience_score)/5,2)}")

Safety score: 42.58
Security score: 73.08
Privacy score: 35.7
Reliability score: 53.18
Resilience score: 53.18
First Trustworthiness score: 53.02
